# Bonus Assignment - Data Wrangling in Action!

This optional homework assignment will cover some of the data programming aspects that we have covered over the extent of our lecture series.

The datasets that will be used are in `.csv` format and can be accessed through the following two urls:

`https://storage.googleapis.com/comp240-stores/used_cars_small.csv`

`https://storage.googleapis.com/comp240-stores/used_cars_small_prices.csv`

The first dataset captures 25 attributes relevant to a large number of used cars that will be sold online, while the second data includes simply the car id and the suggested price for that car.

In [ ]:
import pandas as pd
import numpy as np

**Ex.1**: Import the `used_cars_small.csv` dataset in your notebook, structure it as a Pandas DataFrame and show the first 20 rows.

In [ ]:
#your code goes here
df = pd.read_csv('https://storage.googleapis.com/comp240-stores/used_cars_small.csv')
df.head(20)

In [ ]:
# I saved it to the csv in the current folder so I do not have to download it again
df.to_csv('cars_small.csv', index=False)

In [ ]:
df = pd.read_csv('cars_small.csv')
df.head(20)

**Ex.2**: Now, print only the column names:

In [ ]:
#your code goes here
df.columns.values
# print(*df.columns.values, sep='\n')

**Ex.3**: There are 25 attributes and this is a lot! Perform dimensionality reduction by removing from the DataFrame the `url`, `region_url`, `VIN`, `title_status`, `image_url`, `county`, `lat` and `long` attributes:

In [ ]:
#your code goes here
df = df.drop(columns=['url', 'region_url', 'VIN', 'title_status', 'image_url', 'county', 'lat', 'long'])

**Ex.4**: There should be 6 distinct values (7 with `nan`) for the categorical attribute `condition`, show them:

In [ ]:
#your code goes here
df['condition'].unique()

**Ex.5**: Re-classify the `condition` column so that `fair` values are now `good` and `like new` values are now `excellent`:

In [ ]:
#your code goes here
df_checkpoint = df.copy()
df.loc[df['condition'] == 'fair', 'condition'] = 'good'
df.loc[df['condition'] == 'like new', 'condition'] = 'excellent'
df.head()

**Ex.6** There should be 214 cars where theire `condition` is labeled as `salvage`, `new` or `nan`. Can you show them?

In [ ]:
#your code goes here
df[(df['condition'] == 'salvage') | (df['condition'] == 'new') | (df['condition'] == 'nan')]

**Ex.7**: Can you find the column with the most missing values? If yes, remove this column:

In [ ]:
#your code goes here
column = df.count().idxmin()
print(f'Column with the most NaN values: {column}')
df = df.drop(columns=column)
df.head()

**Ex.8**: Are there any cars that have purchased prior to 1990? If yes, remove those cars:

In [ ]:
#your code goes here
df = df.drop(df[df['year'] < 1990].index)

**Ex.9**: The `year` attribute should feature integer values. Convert it to `int64`:

In [ ]:
#your code goes here
df['year'] = df['year'].astype('Int64')
df.head()

**Ex.10**: Are there any cars that their `odometer` shows more than 2.5 million miles? Should these entries actually be in the dataset? if not, remove them.

In [ ]:
#your code goes here
df[df['odometer'] > 2_500_000][['id', 'year', 'manufacturer', 'odometer']]

In [ ]:
df = df.drop(df[df['odometer'] > 2_500_000].index)

**Ex.11**: How many toyota's are listed?

In [ ]:
#your code goes here
# df_new['manufacturer'][df_new['manufacturer'] == 'toyota'].count()
len(df[df['manufacturer'] == 'toyota'])

**Ex.12**: Let's generalize the above by counting how many cars are listed for all manufacturers:

In [ ]:
#your code goes here
df.groupby('manufacturer')['manufacturer'].count().sort_values(ascending=False)

**Ex.13**: Now, show how many cars are there per vehicle `type`:

In [ ]:
#your code goes here
df.groupby('type')['type'].count().sort_values(ascending=False)

**Ex.14**: Extend the previous to group by `type` and then by `fuel`:

In [ ]:
#your code goes here
result = df.groupby(['type', 'fuel'])['type'].count().to_frame()
result.columns = ['Count']
result.sort_values(by=['type', 'Count'], ascending=[True, False])

**Ex.15**: Import the `used_cars_small_prices.csv` dataset in your notebook, structure it as a Pandas DataFrame and show the first 20 rows.

In [ ]:
#your code goes here
df_prices = pd.read_csv('https://storage.googleapis.com/comp240-stores/used_cars_small_prices.csv')
df_prices.head(20)

In [ ]:
# I saved it to the csv in the current folder so I do not have to download it again
df_prices.to_csv('cars_small_prices.csv', index=False)

In [ ]:
df_prices = pd.read_csv('cars_small_prices.csv')
df_prices.head(20)

In [ ]:
# Checkpoint before merging
df_checkpoint2 = df.copy()

**Ex.16**: Merge the two datasets together so that the cars DataFrame now also captures the price for each car:

In [ ]:
#your code goes here
df_complete = pd.merge(df, df_prices, on='id')
df_complete.head()

**Ex.17**: Are there any cars with a price tag below 100 or above 300K? If yes, remove these values as the law does not permit used vehicles to be sold above 300K.

In [ ]:
#your code goes here
df_complete = df_complete.drop(df_complete[(df_complete['price'] < 100) | (df_complete['price'] > 300_000)].index)

**Ex.18**: Group the cars by manufacturer and produce the mean, median, std, min and max as price as insights for each manufacturer:

In [ ]:
# your code goes here
df_complete.groupby('manufacturer')['price'].agg(['min', 'max', 'mean', 'median', 'std'])

**Ex.19**: Extend the previous to include in the analysis for each manufacturer the stats for also the different `fuel` types:

In [ ]:
#your code goes here
df_complete.groupby(['manufacturer', 'fuel'])['price'].agg(['min', 'max', 'mean', 'median', 'std'])

**Ex.20**: The above seems messy, can you improve visibility with a pivot table?

In [ ]:
#your code goes here
pd.pivot_table(df_complete, values=['price'], index=['manufacturer', 'fuel'], aggfunc=['min', 'max', 'mean', 'median', 'std'], margins=True, margins_name='Total')

# A bit of extra work for the curious

The `prices` dataset is also available in the following url in `json` format.

`https://storage.googleapis.com/comp240-stores/used_cars_small_pricesJSON.json`

Use the `requests` library to download the dataset and then use the `pandas` method `read_json` to import the dataset to your notebook as a DataFrame. After that, you may opt to re-execute Ex.16-20 just to see if things still work.

The goal of this small task is to show that we have achieved *programming portability*. The data ingestion is decoupled from the analysis, meaning that all code written for the analysis is not affected by the why the data is stored.

In [ ]:
#your code goes here
import requests

url = 'https://storage.googleapis.com/comp240-stores/used_cars_small_pricesJSON.json'

r = requests.get(url)

if r:
    df_prices_json = pd.DataFrame(r.json())
    print(df_prices_json.head(20))
else:
    print(r.text)

In [ ]:
# I saved it to the json file in the current directory so I do not have to download it again
df_prices_json.to_json('cars_small_prices.json')

In [ ]:
# df_prices_json = pd.read_json('cars_small_prices.json')
df_complete_json = pd.merge(df, df_prices_json, on='id')
# df_complete_json = df_complete_json.drop(df_complete_json[(df_complete_json['price'] < 100) | (df_complete_json['price'] > 300_000)].index)

In [ ]:
# Drop rows
df_complete_json = df_complete_json.drop(df_complete_json[(df_complete_json['price'] < 100) | (df_complete_json['price'] > 300_000)].index)

In [ ]:
df_complete_json.groupby('manufacturer')['price'].agg(['min', 'max', 'mean', 'median', 'std'])

In [ ]:
#your code goes here
df_complete_json.groupby(['manufacturer', 'fuel'])['price'].agg(['min', 'max', 'mean', 'median', 'std'])

In [ ]:
#your code goes here
pd.pivot_table(df_complete_json, values=['price'], index=['manufacturer', 'fuel'], aggfunc=['min', 'max', 'mean', 'median', 'std'], margins=True, margins_name='Total')